# Lereta Intelligence Agent - ML Models

**Training 3 Machine Learning Models for Tax & Flood Intelligence**

This notebook trains 3 ML models for the Lereta Intelligence Agent:
1. **TAX_DELINQUENCY_PREDICTOR** - Predicts property tax delinquency risk
2. **CLIENT_CHURN_PREDICTOR** - Predicts client churn risk
3. **LOAN_RISK_CLASSIFIER** - Classifies loans by risk level (LOW/MEDIUM/HIGH)

---

## Prerequisites
- Database: `LERETA_INTELLIGENCE`
- Schema: `ANALYTICS`
- Feature views created (V_TAX_DELINQUENCY_FEATURES, V_CLIENT_CHURN_FEATURES, V_LOAN_RISK_FEATURES)
- Packages: `snowflake-ml-python`, `scikit-learn`, `xgboost`, `pandas`


## Setup and Imports


In [ ]:
# Import required libraries
from snowflake.snowpark import Session
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.preprocessing import OrdinalEncoder, StandardScaler
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.registry import Registry
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")


In [ ]:
# Get current session (Snowflake Notebook environment)
session = Session.builder.getOrCreate()

# Set context
session.use_database("LERETA_INTELLIGENCE")
session.use_schema("ANALYTICS")
session.use_warehouse("LERETA_WH")

print("✅ Session configured")
print(f"Database: {session.get_current_database()}")
print(f"Schema: {session.get_current_schema()}")
print(f"Warehouse: {session.get_current_warehouse()}")


In [ ]:
# Initialize Model Registry
registry = Registry(
    session=session,
    database_name="LERETA_INTELLIGENCE",
    schema_name="ANALYTICS"
)

print("✅ Model Registry initialized")


## Model 1: Tax Delinquency Prediction

### Business Problem
Predict which properties are likely to become delinquent on property taxes in the next 90 days. This enables proactive outreach to clients and borrowers to prevent delinquencies.

### Features
- Property characteristics (type, assessed value)
- Tax amount and jurisdiction
- Historical payment patterns
- Days since last payment
- Client service quality score
- Loan characteristics


In [ ]:
# Create training dataset for tax delinquency prediction
tax_delinquency_query = """
SELECT 
    t.tax_record_id,
    -- Target variable
    t.delinquent AS is_delinquent,
    
    -- Property features
    p.property_type,
    p.assessed_value,
    p.flood_zone,
    
    -- Tax features
    t.tax_amount,
    tj.tax_rate,
    tj.jurisdiction_type,
    t.penalty_amount,
    DATEDIFF('day', t.due_date, CURRENT_DATE()) AS days_since_due,
    DATEDIFF('day', COALESCE(t.payment_date, CURRENT_DATE()), CURRENT_DATE()) AS days_since_last_payment,
    
    -- Loan features
    l.loan_type,
    l.loan_amount,
    l.escrow_account,
    l.loan_status,
    
    -- Client features
    c.client_type,
    c.service_quality_score,
    c.client_status,
    
    -- Historical payment behavior
    CASE WHEN t.payment_date IS NULL THEN 1 ELSE 0 END AS has_unpaid_taxes,
    CASE WHEN t.payment_status = 'PAID' THEN 1 ELSE 0 END AS current_paid_status
    
FROM TAX_RECORDS t
JOIN PROPERTIES p ON t.property_id = p.property_id
JOIN LOANS l ON t.loan_id = l.loan_id
JOIN CLIENTS c ON t.client_id = c.client_id
JOIN TAX_JURISDICTIONS tj ON t.jurisdiction_id = tj.jurisdiction_id
WHERE t.tax_year >= YEAR(CURRENT_DATE()) - 2
    AND p.property_status = 'ACTIVE'
    AND l.loan_status = 'ACTIVE'
LIMIT 100000
"""

# Load data into Snowpark DataFrame
tax_delinquency_df = session.sql(tax_delinquency_query)

print(f"Tax Delinquency Dataset Shape: {tax_delinquency_df.count()} rows")
tax_delinquency_df.show(5)


In [ ]:
# Feature engineering for tax delinquency model
categorical_features = ['property_type', 'flood_zone', 'jurisdiction_type', 'loan_type', 'client_type', 'loan_status', 'client_status']
numerical_features = ['assessed_value', 'tax_amount', 'tax_rate', 'penalty_amount', 'days_since_due', 
                      'days_since_last_payment', 'loan_amount', 'service_quality_score', 
                      'has_unpaid_taxes', 'current_paid_status']

# Split data into train/test (80/20)
train_df, test_df = tax_delinquency_df.random_split([0.8, 0.2], seed=42)

print(f"Training set: {train_df.count()} rows")
print(f"Test set: {test_df.count()} rows")

# Build preprocessing pipeline
tax_delinquency_pipeline = Pipeline(
    steps=[
        ('encoder', OrdinalEncoder(input_cols=categorical_features, output_cols=categorical_features)),
        ('scaler', StandardScaler(input_cols=numerical_features, output_cols=numerical_features)),
        ('classifier', RandomForestClassifier(
            input_cols=categorical_features + numerical_features,
            label_cols=['IS_DELINQUENT'],
            n_estimators=100,
            max_depth=10,
            random_state=42
        ))
    ]
)

# Train the model
print("Training Tax Delinquency Prediction Model...")
tax_delinquency_model = tax_delinquency_pipeline.fit(train_df)
print("✅ Model trained successfully!")


In [ ]:
# Evaluate tax delinquency model
predictions_df = tax_delinquency_model.predict(test_df)

# Calculate metrics
print("\n=== Tax Delinquency Model Performance ===")
accuracy = accuracy_score(df=predictions_df, y_true_col_name='IS_DELINQUENT', y_pred_col_name='OUTPUT_IS_DELINQUENT')
precision = precision_score(df=predictions_df, y_true_col_name='IS_DELINQUENT', y_pred_col_name='OUTPUT_IS_DELINQUENT')
recall = recall_score(df=predictions_df, y_true_col_name='IS_DELINQUENT', y_pred_col_name='OUTPUT_IS_DELINQUENT')
f1 = f1_score(df=predictions_df, y_true_col_name='IS_DELINQUENT', y_pred_col_name='OUTPUT_IS_DELINQUENT')

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

# Show sample predictions
predictions_df.select('TAX_RECORD_ID', 'IS_DELINQUENT', 'OUTPUT_IS_DELINQUENT').show(10)


In [ ]:
# Register tax delinquency model to Snowflake Model Registry
session.use_schema("ANALYTICS")

# Create registry
registry = Registry(session=session)

# Register model
model_name = "TAX_DELINQUENCY_PREDICTOR"
model_version = "v1"

print(f"Registering model: {model_name}_{model_version}")
registry.log_model(
    model=tax_delinquency_model,
    model_name=model_name,
    version_name=model_version,
    comment="Predicts property tax delinquency risk using Random Forest. Features: property characteristics, tax amounts, payment history, client metrics."
)

print(f"✅ Model {model_name} version {model_version} registered successfully!")


## Model 2: Client Churn Prediction

### Business Problem
Identify clients (financial institutions) at risk of canceling their Lereta subscriptions. This enables proactive retention efforts and improved customer success.

### Features
- Subscription characteristics (tier, billing cycle, property count)
- Service utilization patterns
- Support ticket volume and satisfaction
- Revenue and transaction trends
- Client profile and status


In [ ]:
# Create training dataset for client churn prediction
session.use_schema("RAW")

client_churn_query = """
WITH client_metrics AS (
    SELECT 
        c.client_id,
        -- Target: Churn indicator (subscription expired and not renewed)
        CASE 
            WHEN s.subscription_status IN ('EXPIRED', 'PENDING_RENEWAL') THEN 1 
            ELSE 0 
        END AS is_churned,
        
        -- Client features
        c.client_type,
        c.service_quality_score,
        c.total_properties,
        c.lifetime_value,
        DATEDIFF('month', c.onboarding_date, CURRENT_DATE()) AS months_as_client,
        
        -- Subscription features
        s.service_type,
        s.subscription_tier,
        s.billing_cycle,
        s.monthly_price,
        s.property_count_limit,
        s.user_licenses,
        s.advanced_analytics,
        DATEDIFF('day', s.start_date, COALESCE(s.end_date, CURRENT_DATE())) AS subscription_duration_days,
        
        -- Support metrics (aggregated)
        COUNT(DISTINCT st.ticket_id) AS total_support_tickets,
        AVG(st.satisfaction_rating) AS avg_satisfaction_rating,
        AVG(st.resolution_time_hours) AS avg_resolution_time,
        SUM(CASE WHEN st.ticket_status = 'OPEN' THEN 1 ELSE 0 END) AS open_tickets,
        
        -- Revenue metrics
        COUNT(DISTINCT t.transaction_id) AS total_transactions,
        SUM(t.total_amount) AS total_revenue,
        AVG(t.total_amount) AS avg_transaction_amount
        
    FROM CLIENTS c
    LEFT JOIN SERVICE_SUBSCRIPTIONS s ON c.client_id = s.client_id
    LEFT JOIN SUPPORT_TICKETS st ON c.client_id = st.client_id
    LEFT JOIN TRANSACTIONS t ON c.client_id = t.client_id
    WHERE c.client_status IN ('ACTIVE', 'SUSPENDED')
        AND s.subscription_id IS NOT NULL
    GROUP BY 
        c.client_id, c.client_type, c.service_quality_score, c.total_properties, 
        c.lifetime_value, c.onboarding_date, s.service_type, s.subscription_tier,
        s.billing_cycle, s.monthly_price, s.property_count_limit, s.user_licenses,
        s.advanced_analytics, s.subscription_status, s.start_date, s.end_date
)
SELECT * FROM client_metrics
LIMIT 50000
"""

client_churn_df = session.sql(client_churn_query)

print(f"Client Churn Dataset Shape: {client_churn_df.count()} rows")
client_churn_df.show(5)


In [ ]:
# Feature engineering for churn prediction
categorical_features_churn = ['CLIENT_TYPE', 'SERVICE_TYPE', 'SUBSCRIPTION_TIER', 'BILLING_CYCLE']
numerical_features_churn = [
    'SERVICE_QUALITY_SCORE', 'TOTAL_PROPERTIES', 'LIFETIME_VALUE', 'MONTHS_AS_CLIENT',
    'MONTHLY_PRICE', 'PROPERTY_COUNT_LIMIT', 'USER_LICENSES', 'SUBSCRIPTION_DURATION_DAYS',
    'TOTAL_SUPPORT_TICKETS', 'AVG_SATISFACTION_RATING', 'AVG_RESOLUTION_TIME', 'OPEN_TICKETS',
    'TOTAL_TRANSACTIONS', 'TOTAL_REVENUE', 'AVG_TRANSACTION_AMOUNT'
]

# Split data
train_churn_df, test_churn_df = client_churn_df.random_split([0.8, 0.2], seed=42)

print(f"Training set: {train_churn_df.count()} rows")
print(f"Test set: {test_churn_df.count()} rows")

# Build churn prediction pipeline with XGBoost
client_churn_pipeline = Pipeline(
    steps=[
        ('encoder', OrdinalEncoder(input_cols=categorical_features_churn, output_cols=categorical_features_churn)),
        ('scaler', StandardScaler(input_cols=numerical_features_churn, output_cols=numerical_features_churn)),
        ('classifier', XGBClassifier(
            input_cols=categorical_features_churn + numerical_features_churn,
            label_cols=['IS_CHURNED'],
            n_estimators=150,
            max_depth=8,
            learning_rate=0.1,
            random_state=42
        ))
    ]
)

# Train the model
print("Training Client Churn Prediction Model...")
client_churn_model = client_churn_pipeline.fit(train_churn_df)
print("✅ Model trained successfully!")


In [ ]:
# Evaluate churn model
churn_predictions_df = client_churn_model.predict(test_churn_df)

print("\n=== Client Churn Model Performance ===")
accuracy_churn = accuracy_score(df=churn_predictions_df, y_true_col_name='IS_CHURNED', y_pred_col_name='OUTPUT_IS_CHURNED')
precision_churn = precision_score(df=churn_predictions_df, y_true_col_name='IS_CHURNED', y_pred_col_name='OUTPUT_IS_CHURNED')
recall_churn = recall_score(df=churn_predictions_df, y_true_col_name='IS_CHURNED', y_pred_col_name='OUTPUT_IS_CHURNED')
f1_churn = f1_score(df=churn_predictions_df, y_true_col_name='IS_CHURNED', y_pred_col_name='OUTPUT_IS_CHURNED')

print(f"Accuracy:  {accuracy_churn:.4f}")
print(f"Precision: {precision_churn:.4f}")
print(f"Recall:    {recall_churn:.4f}")
print(f"F1 Score:  {f1_churn:.4f}")

churn_predictions_df.select('CLIENT_ID', 'IS_CHURNED', 'OUTPUT_IS_CHURNED').show(10)


In [ ]:
# Register churn model
session.use_schema("ANALYTICS")

model_name_churn = "CLIENT_CHURN_PREDICTOR"
model_version_churn = "v1"

print(f"Registering model: {model_name_churn}_{model_version_churn}")
registry.log_model(
    model=client_churn_model,
    model_name=model_name_churn,
    version_name=model_version_churn,
    comment="Predicts client churn risk using XGBoost. Features: subscription metrics, support satisfaction, revenue trends, service utilization."
)

print(f"✅ Model {model_name_churn} version {model_version_churn} registered successfully!")


## Model 3: Loan Risk Classification

### Business Problem
Classify loans into risk categories (Low, Medium, High) based on tax compliance, flood zone risks, and property characteristics. This helps prioritize monitoring efforts and identify high-risk portfolios.

### Features
- Flood zone and insurance requirements
- Tax payment history and delinquency status
- Loan-to-value ratio
- Property type and assessed value
- Escrow account status
- Jurisdiction tax rates


In [ ]:
# Create training dataset for loan risk classification
session.use_schema("RAW")

loan_risk_query = """
WITH loan_features AS (
    SELECT 
        l.loan_id,
        
        -- Target variable: Risk Level (engineered from multiple factors)
        CASE 
            WHEN (p.flood_zone IN ('AE', 'A', 'VE') AND fc.insurance_required = FALSE) 
                OR (t.delinquent = TRUE AND t.penalty_amount > 500) 
                OR (l.loan_status = 'FORECLOSED') THEN 'HIGH'
            WHEN (p.flood_zone IN ('AE', 'A', 'VE') AND fc.insurance_required = TRUE) 
                OR (t.payment_status = 'PENDING' AND DATEDIFF('day', t.due_date, CURRENT_DATE()) > 30)
                OR (l.escrow_account = FALSE AND t.delinquent = TRUE) THEN 'MEDIUM'
            ELSE 'LOW'
        END AS risk_level,
        
        -- Loan features
        l.loan_type,
        l.loan_amount,
        l.loan_status,
        l.escrow_account,
        DATEDIFF('month', l.loan_date, CURRENT_DATE()) AS loan_age_months,
        (l.loan_amount / NULLIF(p.assessed_value, 0))::DOUBLE AS loan_to_value_ratio,
        
        -- Property features
        p.property_type,
        p.assessed_value,
        p.flood_zone,
        p.property_state,
        
        -- Flood certification features
        fc.insurance_required,
        fc.life_of_loan_tracking,
        CASE WHEN fz.risk_level = 'HIGH_RISK' THEN 1 ELSE 0 END AS high_flood_risk,
        
        -- Tax features
        t.tax_amount,
        t.delinquent,
        t.penalty_amount,
        tj.tax_rate,
        tj.jurisdiction_type,
        CASE WHEN t.payment_status = 'PAID' THEN 1 ELSE 0 END AS tax_paid_on_time,
        DATEDIFF('day', t.due_date, COALESCE(t.payment_date, CURRENT_DATE())) AS days_payment_delay,
        
        -- Client features
        c.client_type,
        c.service_quality_score
        
    FROM LOANS l
    JOIN PROPERTIES p ON l.property_id = p.property_id
    LEFT JOIN FLOOD_CERTIFICATIONS fc ON l.loan_id = fc.loan_id
    LEFT JOIN FLOOD_ZONES fz ON p.flood_zone = fz.zone_id
    LEFT JOIN TAX_RECORDS t ON l.loan_id = t.loan_id
    LEFT JOIN TAX_JURISDICTIONS tj ON t.jurisdiction_id = tj.jurisdiction_id
    LEFT JOIN CLIENTS c ON l.client_id = c.client_id
    WHERE l.loan_status IN ('ACTIVE', 'PAID_OFF', 'FORECLOSED')
        AND p.property_status = 'ACTIVE'
)
SELECT * FROM loan_features
WHERE risk_level IS NOT NULL
LIMIT 100000
"""

loan_risk_df = session.sql(loan_risk_query)

print(f"Loan Risk Dataset Shape: {loan_risk_df.count()} rows")
loan_risk_df.show(5)


In [ ]:
# Feature engineering for loan risk classification
categorical_features_risk = [
    'LOAN_TYPE', 'LOAN_STATUS', 'PROPERTY_TYPE', 'FLOOD_ZONE', 
    'PROPERTY_STATE', 'JURISDICTION_TYPE', 'CLIENT_TYPE'
]
numerical_features_risk = [
    'LOAN_AMOUNT', 'LOAN_AGE_MONTHS', 'LOAN_TO_VALUE_RATIO', 'ASSESSED_VALUE',
    'HIGH_FLOOD_RISK', 'TAX_AMOUNT', 'PENALTY_AMOUNT', 'TAX_RATE',
    'TAX_PAID_ON_TIME', 'DAYS_PAYMENT_DELAY', 'SERVICE_QUALITY_SCORE'
]

# Split data
train_risk_df, test_risk_df = loan_risk_df.random_split([0.8, 0.2], seed=42)

print(f"Training set: {train_risk_df.count()} rows")
print(f"Test set: {test_risk_df.count()} rows")

# Build loan risk classification pipeline
loan_risk_pipeline = Pipeline(
    steps=[
        ('encoder', OrdinalEncoder(input_cols=categorical_features_risk, output_cols=categorical_features_risk)),
        ('scaler', StandardScaler(input_cols=numerical_features_risk, output_cols=numerical_features_risk)),
        ('classifier', RandomForestClassifier(
            input_cols=categorical_features_risk + numerical_features_risk,
            label_cols=['RISK_LEVEL'],
            n_estimators=120,
            max_depth=12,
            random_state=42
        ))
    ]
)

# Train the model
print("Training Loan Risk Classification Model...")
loan_risk_model = loan_risk_pipeline.fit(train_risk_df)
print("✅ Model trained successfully!")


In [ ]:
# Evaluate loan risk model
risk_predictions_df = loan_risk_model.predict(test_risk_df)

print("\n=== Loan Risk Classification Model Performance ===")
accuracy_risk = accuracy_score(df=risk_predictions_df, y_true_col_name='RISK_LEVEL', y_pred_col_name='OUTPUT_RISK_LEVEL')

print(f"Accuracy: {accuracy_risk:.4f}")

# Show sample predictions
risk_predictions_df.select('LOAN_ID', 'RISK_LEVEL', 'OUTPUT_RISK_LEVEL', 'FLOOD_ZONE', 'DELINQUENT').show(10)


In [ ]:
# Register loan risk model
session.use_schema("ANALYTICS")

model_name_risk = "LOAN_RISK_CLASSIFIER"
model_version_risk = "v1"

print(f"Registering model: {model_name_risk}_{model_version_risk}")
registry.log_model(
    model=loan_risk_model,
    model_name=model_name_risk,
    version_name=model_version_risk,
    comment="Classifies loans into LOW/MEDIUM/HIGH risk categories using Random Forest. Features: flood zones, tax compliance, LTV ratio, property characteristics."
)

print(f"✅ Model {model_name_risk} version {model_version_risk} registered successfully!")


## Summary and Next Steps

### Models Created

1. **TAX_DELINQUENCY_PREDICTOR** (Random Forest)
   - Predicts property tax delinquency risk
   - Use case: Proactive client alerts, portfolio risk management

2. **CLIENT_CHURN_PREDICTOR** (XGBoost)
   - Predicts client subscription cancellation risk
   - Use case: Customer retention, account management prioritization

3. **LOAN_RISK_CLASSIFIER** (Random Forest)
   - Classifies loans into risk levels (LOW/MEDIUM/HIGH)
   - Use case: Portfolio risk assessment, monitoring prioritization

### Integration with AI Agent

All models are registered in the Snowflake Model Registry and can be:
- Called via SQL UDFs (created in file 07)
- Integrated with the Lereta Intelligence Agent
- Used for batch scoring and real-time predictions
- Monitored for performance drift

### Next Steps

1. Execute **file 07** (Python wrappers) to create SQL UDFs for each model
2. Execute **file 08** (AI Agent creation) to integrate models with the agent
3. Test model predictions through the AI Agent interface
4. Monitor model performance and retrain as needed


In [ ]:
# Verify all models in registry
print("\n=== Registered Models ===")
registry.show_models()

# Close session
session.close()
print("\n✅ All models trained and registered successfully!")
